In [1]:
from math import ceil

import pandas as pd
import numpy as np

import os.path as osp

In [2]:
data = pd.read_csv(osp.join('..', 'Data', 'Groceries 1.csv'))
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Member_number    38765 non-null  int64 
 1   Date             38765 non-null  object
 2   itemDescription  38765 non-null  object
dtypes: int64(1), object(2)
memory usage: 908.7+ KB


In [3]:
transactions = data.groupby(['Member_number', 'Date'])\
                   .aggregate(set)\
                   .reset_index()
transactions['Tid'] = transactions['Member_number'].astype('str') + '-' + transactions['Date']
transactions.drop(['Member_number', 'Date'], axis=1, inplace=True)
transactions

,itemDescription,Tid
0,"{yogurt, whole milk, semi-finished bread, saus...",1000-15-03-2015
1,"{whole milk, pastry, salty snack}",1000-24-06-2014
2,"{canned beer, misc. beverages}",1000-24-07-2015
3,"{sausage, hygiene articles}",1000-25-11-2015
4,"{pickled vegetables, soda}",1000-27-05-2015
...,...,...
14958,"{napkins, kitchen towels, other vegetables, tr...",4999-24-01-2015
14959,"{bottled water, herbs}",4999-26-12-2015
14960,"{onions, fruit/vegetable juice}",5000-09-03-2014
14961,"{root vegetables, semi-finished bread, soda}",5000-10-02-2015


In [4]:
from eclat_algo import eclat
# from pyECLAT import ECLAT
from mlxtend.frequent_patterns import fpgrowth, apriori
from mlxtend.preprocessing import TransactionEncoder

In [5]:
transformed_df = data.copy(deep=True)
transformed_df['Tid'] = transformed_df['Member_number'].astype('str') + '-' + transformed_df['Date']
transformed_df.drop(['Member_number', 'Date'], axis=1, inplace=True)

In [6]:
# max_items = transformed_df.drop_duplicates().groupby('Tid').count().max().iloc[0]
# transactions = transformed_df.drop_duplicates().groupby('Tid').aggregate(list).reset_index()

# item_array = np.ndarray((len(transformed_df['Tid'].unique()), max_items), dtype=object)
# for i, row in transactions.iterrows():
#     if (max_items - len(row['itemDescription'])) != 0 :
#         transact = row['itemDescription'] + [None] * (max_items - len(row['itemDescription']))
#     else: transact = row['itemDescription']
#     item_array[i] = transact

In [28]:
minSup = .01
minConf = .5

no_transacts = len(transformed_df['Tid'].unique())
minFreq = ceil(no_transacts * minSup)
minFreq

150

In [30]:
transactions = transformed_df.drop_duplicates().groupby('Tid').aggregate(list).reset_index()

encoder = TransactionEncoder()
encoder.fit(transactions['itemDescription'].values)
array = encoder.transform(transactions['itemDescription'].values)

test_result = fpgrowth(pd.DataFrame(array, columns=encoder.columns_), minSup, use_colnames=True)

In [31]:
my_results = eclat(transformed_df, minSup, 'itemDescription')

In [36]:
# pd.concat([my_results['frequencies'].apply(lambda s: s / no_transacts), my_results['itemsets']],
#           axis=1).sort_values('frequencies')
sorted_results = my_results.sort_values('frequencies')
sorted_results

,itemsets,frequencies
0,[soft cheese],150
1,[processed cheese],152
2,[red/blush wine],157
3,[herbs],158
66,"[rolls/buns, other vegetables]",158
...,...,...
59,[yogurt],1285
60,[soda],1453
61,[rolls/buns],1646
62,[other vegetables],1827


In [37]:
sorted_test = test_result.sort_values('support')
sorted_test

,support,itemsets
52,0.010025,(soft cheese)
47,0.010158,(processed cheese)
26,0.010493,(red/blush wine)
37,0.010559,(herbs)
67,0.010559,"(other vegetables, rolls/buns)"
...,...,...
1,0.085879,(yogurt)
8,0.097106,(soda)
11,0.110005,(rolls/buns)
15,0.122101,(other vegetables)
